In [3]:
from sqlalchemy import create_engine
import pandas as pd

In [4]:
# Load the existing CSV file
existing_df = pd.read_csv('2023_data_cleanedcopy.csv')

# Load the additional CSV file with airport coordinates
coordinates_df = pd.read_csv('output.csv')

# Ensure no duplicate airports in coordinates_df
coordinates_df = coordinates_df.drop_duplicates(subset=['airport'])

# Create empty columns in the existing DataFrame for longitude and latitude
existing_df['longitude'] = None
existing_df['latitude'] = None



In [5]:
# Create a dictionary for quick lookup of coordinates
coordinates_dict = coordinates_df.set_index('airport')[['longitude', 'latitude']].to_dict('index')

# Iterate through each row in the existing DataFrame
for index, row in existing_df.iterrows():
    airport_code = row['airport']
    if airport_code in coordinates_dict:
        existing_df.at[index, 'longitude'] = coordinates_dict[airport_code]['longitude']
        existing_df.at[index, 'latitude'] = coordinates_dict[airport_code]['latitude']

# Display the updated DataFrame to verify longitude and latitude are added
print("Updated DataFrame with longitude and latitude:")
print(existing_df.head())

Updated DataFrame with longitude and latitude:
   month       carrier_name airport                            city  \
0     12  Endeavor Air Inc.     ABE  Allentown/Bethlehem/Easton, PA   
1     12  Endeavor Air Inc.     AEX                  Alexandria, LA   
2     12  Endeavor Air Inc.     AGS                     Augusta, GA   
3     12  Endeavor Air Inc.     ALB                      Albany, NY   
4     12  Endeavor Air Inc.     ATL                     Atlanta, GA   

   total_arrivals  total_delays_ct  carrier_ct  weather_ct  nat_air_sys_ct  \
0            72.0              5.0        2.46        1.00            0.73   
1            62.0              7.0        4.25        0.00            1.00   
2            95.0             10.0        5.94        0.00            1.06   
3            23.0              2.0        0.56        0.00            0.00   
4          2111.0            256.0       76.88        8.75           52.43   

   security_ct  ...  flight_cancelled  flight_diverted  t

In [6]:
# Save the updated DataFrame to a new CSV file
existing_df.to_csv('long_lat_test.csv', index=False)

In [7]:
# Confirm the file is saved and display a message
print("Updated DataFrame saved to 'updated_file.csv'")

Updated DataFrame saved to 'updated_file.csv'


In [5]:
import csv
import json
from collections import OrderedDict

li = []
with open('clean_plots.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        latitude = row[0]
        longitude = row[1]
        total_delays_ct = row[2]

        d = OrderedDict()
        d['type'] = 'Feature'
        d['geometry'] = {
            'type': 'Point',
            'coordinates': [float(latitude), float(longitude)]
        }
        d['properties'] = {
            'Total Delays': total_delays_ct,
            'Airport': airport
        }
        li.append(d)

d = OrderedDict()
d['type'] = 'FeatureCollection'
d['features'] = li
with open('GeoObs.json', 'w') as f:
    f.write(json.dumps(d, sort_keys=False, indent=4))

ValueError: could not convert string to float: 'airport'

In [2]:
import csv, json
from geojson import Feature, FeatureCollection, Point

features = []
with open('long_lat_test.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for latitude, longitude, total_delays_ct in reader:
        latitude, longitude = map(float, (latitude, longitude))
        features.append(
            Feature(
                geometry = Point((longitude, latitude)),
                properties = {
                    'Total Delays': total_delays_ct
                }
            )
        )

collection = FeatureCollection(features)
with open("GeoObs.json", "w") as f:
    f.write('%s' % collection)

ValueError: too many values to unpack (expected 3)